In [11]:
from dotenv import load_dotenv

load_dotenv()

True

In [12]:
from langchain.llms import OpenAI

In [13]:
llm = OpenAI(model="gpt-3.5-turbo")
result = llm.invoke("List 8 wonders of the word")
print(result)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

### Streaming and Batching


In [29]:
for chunk in llm.stream("8 Wonders of World"):
    print(chunk.content)

1. Great Wall of China
2. Chichen Itza
3.
 Christ the Redeemer
4. Colosseum
5. Machu Picchu
6. Petra
7. Taj Mahal
8. Great Pyramid of Giza


In [30]:
result = llm.batch(
    [
        "What's 2 +2 ?",
        "What's 34.9 + 23.78?"
    ]
)
for _result in result:
    print(_result.content)

4
58.68


### System & Human Message

In [36]:
from langchain_core.messages import SystemMessage, HumanMessage

messages = [
    SystemMessage(content="You are Virat Kohli"),
    HumanMessage(content="Which crickrt bat manufacturer you are associated with?")
]

result = llm.invoke(messages)
print(result.content)

/home/arslanhaider/.local/share/virtualenvs/generative-ai-cfm5DT86/lib/python3.12/site-packages/langchain_google_genai/chat_models.py:352: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


MRF


### Prompt Template

In [37]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "tell me a {objective} joke about {content}"
)

filled_prompt = prompt_template.format(objective="funny", content="robots")

response = llm.invoke(filled_prompt)
response.content

'What do you call a robot that loves to dance?\n\nA boogie-bot!'

### Chat Prompt Template

In [38]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# Defining a chat template with various roles
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a helpful AI Mathematics teacher bot. Your name is Bob."),
        HumanMessage(content="Hello, How are you doing?"),
        HumanMessage(content=" what is a + b square? give an example from day tp day life")
    ]
)

formatted_messages = chat_template.format_messages()

for message in formatted_messages:
    result = llm.invoke(message.content)
    print(result.content)

Hello, I am Bob, a virtual Mathematics teacher. I am here to help you with your Math homework, practice problems, or any other math-related questions you may have. Feel free to ask me anything you need help with, and I will do my best to explain it to you in a clear and concise way.

Here are some of the things I can help you with:

* **Homework help:** I can help you understand your homework problems and provide step-by-step solutions.
* **Practice problems:** I can generate practice problems for you to work on, and provide feedback on your answers.
* **Math concepts:** I can explain math concepts to you in a clear and concise way.
* **Test preparation:** I can help you prepare for math tests by providing practice problems and reviewing key concepts.

I am still under development, but I am learning more every day. I am always looking for ways to improve my knowledge and skills, so that I can be the best possible math teacher for you.

Please don't hesitate to ask me any math-related q

### Output Parsers

Simple Json Parser

In [47]:
from langchain.output_parsers.json import SimpleJsonOutputParser

json_prompt = PromptTemplate.from_template(
    "Return a json object with keys `birth_place` and `birth_date` that answers the following question {content}"
)

json_parser = SimpleJsonOutputParser()

def query_llm(content):
  """Queries the LLM for birth details."""
  return llm.invoke(content)


# create chain with prompt, LLM and parser

json_chain = prompt_template | query_llm | json_parser

# Stream through the result
result = list(json_chain.stream({"content": "When and where elon musk & bill gates are born?", "objective": "Provide birth details"}))

print(result)

[]
